In [ ]:
import requests
import json
import datetime

In [48]:
PERF_DIR = "../../data/perf"

In [49]:
base_url = "http://le-terror:9090/api/v1/query_range"
max_data_points = 10_000

In [50]:
def cpu_usage_merge_fn(accumulator, query_result):
    for container in query_result["data"]["result"]:
        container_name = container["metric"]["name"]

        if container_name not in accumulator:
            accumulator[container_name] = []

        for sample in container["values"]:
            accumulator[container_name].append(sample)


def mem_usage_merge_fn(accumulator, query_result):
    for container in query_result["data"]["result"]:
        container_name = container["metric"]["name"]

        if container_name not in accumulator:
            accumulator[container_name] = []

        for sample in container["values"]:
            accumulator[container_name].append(sample)


def disk_usage_merge_fn(accumulator, query_result):
    for device in query_result["data"]["result"]:
        device_name = device["metric"]["device"]

        if device_name not in accumulator:
            accumulator[device_name] = []

        for sample in device["values"]:
            accumulator[device_name].append(sample)


def gpu_usage_merge_fn(accumulator, query_result):
    for gpu in query_result["data"]["result"]:
        gpu_name = gpu["metric"]["uuid"]

        if gpu_name not in accumulator:
            accumulator[gpu_name] = []

        for sample in gpu["values"]:
            accumulator[gpu_name].append(sample)


def gpu_mem_usage_merge_fn(accumulator, query_result):
    for gpu in query_result["data"]["result"]:
        gpu_name = gpu["metric"]["uuid"]

        if gpu_name not in accumulator:
            accumulator[gpu_name] = []

        for sample in gpu["values"]:
            accumulator[gpu_name].append(sample)


def gpu_power_draw_merge_fn(accumulator, query_result):
    for gpu in query_result["data"]["result"]:
        gpu_name = gpu["metric"]["uuid"]

        if gpu_name not in accumulator:
            accumulator[gpu_name] = []

        for sample in gpu["values"]:
            accumulator[gpu_name].append(sample)


queries = {
    "cpu_usage": {
        "query": "container_cpu_usage_seconds_total{image=~\"getmeili/meilisearch:latest|postgres-contrib|bachelor-script\"}",
        "merge_fn": cpu_usage_merge_fn
    },
    "mem_usage": {
        "query": "container_memory_usage_bytes{image=~\"getmeili/meilisearch:latest|postgres-contrib|bachelor-script\"}",
        "merge_fn": mem_usage_merge_fn
    },
    "disk_usage": {
        "query": "container_fs_usage_bytes{device=\"/dev/nvme0n1p1\", id=\"/\"}",
        "merge_fn": disk_usage_merge_fn
    },
    "gpu_usage": {
        "query": "nvidia_smi_utilization_gpu_ratio{uuid=\"5e80926d-27b6-a1dc-cc4f-64f033face7f\"}",
        "merge_fn": gpu_usage_merge_fn
    },
    "gpu_mem_usage": {
        "query": "nvidia_smi_memory_used_bytes{uuid=\"5e80926d-27b6-a1dc-cc4f-64f033face7f\"}",
        "merge_fn": gpu_mem_usage_merge_fn
    },
    "gpu_power_draw": {
        "query": "nvidia_smi_power_draw_watts{uuid=\"5e80926d-27b6-a1dc-cc4f-64f033face7f\"}",
        "merge_fn": gpu_power_draw_merge_fn
    }
}

In [51]:
time_chunks = [
    (
        "2025-02-26T20:30:00+01:00",
        "2025-02-27T15:50:00+01:00"
    )
]

step = "1"

In [52]:
for time_chunk in time_chunks:
    chunk_start, chunk_end = time_chunk

    start_parsed = datetime.datetime.fromisoformat(chunk_start)
    end_parsed = datetime.datetime.fromisoformat(chunk_end)

    start = start_parsed.timestamp()
    end = end_parsed.timestamp()

    for query_name, query_details in queries.items():
        accumulator = {}
        cursor = start

        while cursor < end:
            real_end = min(cursor + max_data_points, end)

            params = {
                "query": query_details["query"],
                "start": cursor,
                "end": real_end,
                "step": step
            }

            response = requests.get(base_url, params=params)
            response_json = response.json()

            merge_fn = query_details["merge_fn"]
            merge_fn(accumulator, response_json)

            cursor += max_data_points

        with open(f"{PERF_DIR}/{query_name}.json", "w") as file:
            json.dump(accumulator, file, indent=2)